## ME 481: Whole Body Biomechanics - Balance Post-Lab Individual Assignment

In this post-lab assignment, you will analyze the force plate data you collected during the balance lab. You'll apply the signal processing techniques learned in the pre-lab, calculate center of pressure (CoP) trajectories, and quantify postural control using standard balance metrics.

**How to use this notebook:**
- Each team member will analyze their own data individually in this notebook (Questions 1-4).
- Each section has some code for you to complete. Each of these sections is denoted by "#TODO"
- At the end of each section, complete the discussion questions
- After completing this notebook individually, you will complete the group notebook together as a team.
- Each team member should submit this completed notebook individually (**as HTML and .ipynb with your NetID appended to the filename**) to Canvas. One team member should submit the group assignment notebook (**as HTML and .ipynb**) to Canvas.

**Assignment Structure:**
- **Setup:** Imports and Mounting Google Drive
- **Question 1:** Load Data and Convert Voltages to Forces and Moments
- **Question 2:** Filter Force and Moment Signals
- **Question 3:** Calculate and Plot Center of Pressure (CoP)
- **Question 4:** Calculate CoP Balance Metrics
- **Export Notebook** as HTML for Submission

**Course Context:** This lab demonstrates a complete workflow for biomechanical data analysis: 
- Raw data acquisition
- Signal processing
- Analysis
- Interpretation
These skills are fundamental to experimental biomechanics research.

## Lab Overview and Learning Objectives

### What Did We Measure in the Lab?

In the lab, you performed tandem stance (heel-to-toe) balance tests under two conditions: **Eyes Open (EO)** and **Eyes Closed (EC)**. The AMTI force plate recorded six channels of raw voltage data representing forces (Fx, Fy, Fz) and moments (Mx, My, Mz) applied to its surface. By removing visual feedback in the EC condition, we isolated the contributions of vestibular and proprioceptive systems to postural control.

### By the end of this assignment, you will be able to:

1. **Transform raw sensor data** - Apply calibration matrices to convert voltage signals into biomechanically meaningful forces and moments
2. **Apply signal processing** - Use spectral analysis and filtering to remove noise while preserving physiological signals
3. **Calculate biomechanical metrics** - Compute center of pressure (CoP) trajectories from force plate data
4. **Quantify postural control** - Calculate and interpret standard balance metrics used in clinical and research settings
5. **Compare experimental conditions** - Evaluate how sensory disruption affects postural stability

## Setup: Imports and Mounting Google Drive

### A. Import Required Libraries

Run the cell below to import all necessary libraries for data analysis, signal processing, and visualization.

In [1]:
# Standard library imports
import os  # For interacting with the operating system (e.g., file paths)
import requests  # For making HTTP requests
import nbformat  # For working with Jupyter Notebook file formats
from nbconvert import HTMLExporter # For converting Jupyter Notebooks to HTML

# Third-party library imports
import numpy as np  # For numerical computations and array operations
import pandas as pd  # For data manipulation and analysis using DataFrames
from scipy.signal import welch, butter, filtfilt  # For signal processing (spectral density, filtering)
import matplotlib.pyplot as plt  # For creating plots and visualizations

# Jupyter-specific magic commands
%matplotlib inline

# Set default plot parameters
plt.style.use('default')
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 'large'
plt.rcParams['ytick.labelsize'] = 'large'
plt.rcParams['font.family'] = 'Sans'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.grid'] = True

### B. Connect to Google Drive and Set Working Directory

Mount your Google Drive and set your working directory to your lab folder where your force plate data is stored.

In [ ]:
# Connect to your Google Drive (will ask you to log in and give access)
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
# Set your working directory to your lab folder
# Replace with your group's folder path
lab_folder = '/content/drive/MyDrive/ME481-BalanceLab'
os.chdir(lab_folder)
print(f"✓ Working directory set to: {os.getcwd()}")

---
## **Question 1: Convert Voltages to Forces and Moments**

Convert the measured voltages into forces and moments and create a 6x1 subplot showing forces and moments for your eyes-open (EO) and eyes-closed (EC) data. Each subplot should have a single force/moment on the y-axis and time on the x-axis. Include a legend to differentiate the EO and EC conditions.

### Instructions: Applying the Calibration Matrix

To convert measured voltages into forces and moments, you need the inverted sensitivity matrix, $B$, the gain value from the amplifier, $G$, and the channel input voltage, $V_0$. For this specific force plate and amplifier:

$$
B = \begin{bmatrix}
2.9007     & 0.0200     & -0.0009     & -0.0253   & -0.0085      & 0.0090    \\
-0.0067     & 2.9024   & -0.0520     & -0.0366   & -0.0149     & -0.0341   \\
0.0046   & -0.0229     & 11.4206   & -0.0055     & 0.0055    & 0.0026      \\
-0.0019     & 0.0035     & -0.0067     & 1.4559    & -0.0053     & -0.0028    \\
0.0036     & 0.0011     & -0.0067     & 0.0018    & 1.1475     & -0.0008    \\
0.0037     & 0.0145   & -0.0032   & 0.0006   & 0.0076     & 0.6188
\end{bmatrix}
$$

```python
# Python code to create the matrix
B = np.array([
    [2.9007,  0.0200, -0.0009, -0.0253, -0.0085,  0.0090],
    [-0.0067, 2.9024, -0.0520, -0.0366, -0.0149, -0.0341],
    [0.0046, -0.0229, 11.4206, -0.0055,  0.0055,  0.0026],
    [-0.0019,  0.0035, -0.0067,  1.4559, -0.0053, -0.0028],
    [0.0036,  0.0011, -0.0067,  0.0018,  1.1475, -0.0008],
    [0.0037,  0.0145, -0.0032,  0.0006,  0.0076,  0.6188]
])
```

$$
G = 4000
$$

$$
V_0 = 10 \text{ volts}
$$

Assuming at a certain time-point $t_1$ we have a 1×6 voltage row vector $\vec{V_1} = (V_{Fx}, V_{Fy}, \ldots, V_{Mz})$, we can calculate the corresponding forces and moments, $\vec{Y_1} =(F_{x}, F_{y}, \ldots, M_{z})$ using:

$$
\vec{Y_1} = \frac{10^6}{GV_0}\vec{V_1}\boldsymbol{B^T}
$$


Start by loading your data:

In [ ]:
# 1. Load your EO and EC data files
# Set column names
cols = ['Time', 'VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz']

# Load EO and EC data
# In the Colab file tree, you can right-click on your data files and select "Copy path" or "Copy relative path" to get the correct path
eo_data = # TODO add code to load EO data using pd.read_csv() with the correct file path and column names
ec_data = # TODO add code to load EC data using pd.read_csv() with the correct file path and column names

# Inspect the first few rows of the data
eo_data.head()
ec_data.head()

Now convert voltages to forces/moments

In [ ]:
# Your code goes here

# 2. Define the calibration matrix B, gain G, and input voltage
# Inverted sensitivity matrix, B
B = np.array([
    [2.9007,  0.0200, -0.0009, -0.0253, -0.0085,  0.0090],
    [-0.0067, 2.9024, -0.0520, -0.0366, -0.0149, -0.0341],
    [0.0046, -0.0229, 11.4206, -0.0055,  0.0055,  0.0026],
    [-0.0019,  0.0035, -0.0067,  1.4559, -0.0053, -0.0028],
    [0.0036,  0.0011, -0.0067,  0.0018,  1.1475, -0.0008],
    [0.0037,  0.0145, -0.0032,  0.0006,  0.0076,  0.6188]
])

G = 4000  # Gain
V_0 = 10  # Input voltage
CF = 1e6/(G*V_0)  # Conversion Factor

# 3. Convert voltages to forces/moments using the equation above
V_EO = eo_data[['VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz']].to_numpy()
V_EC = ec_data[['VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz']].to_numpy()

F_EO = #TODO add code to compute F_EO using the conversion factor CF, calibration matrix B, and voltage data V_EO
F_EC = #TODO add code to compute F_EC using the conversion factor CF, calibration matrix B, and voltage data V_EC

# Assign back to DataFrames
eo_data[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']] = F_EO
ec_data[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']] = F_EC

# Inspect the first few rows of the converted data
eo_data.head()
ec_data.head()

Now let's plot the calculated forces/moments

In [ ]:
# 4. Create a 6x1 subplot comparing EO and EC conditions
fig, axes = plt.subplots(6, 1, figsize=(10, 8), dpi=100, sharex=True)
cols = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']
force_labels = ['$F_x$ [N]', '$F_y$ [N]', '$F_z$ [N]',
                '$M_x$ [Nm]', '$M_y$ [Nm]', '$M_z$ [Nm]']

for i, label in enumerate(cols):
    axes[i].plot(#TODO, #TODO, label='EO', color='tab:blue')
    axes[i].plot(#TODO, #TODO, label='EC', color='tab:orange')
    axes[i].set_title(f'Comparison of {label} between EO and EC')
    axes[i].set_ylabel(force_labels[i])
    if i == 0:
        axes[i].legend(loc='upper right')
plt.xlabel('Time [s]')
plt.tight_layout()
plt.show()

### Discussion Questions:

1. How would an error in the gain $G$ or input voltage $V_0$ change the magnitude of your forces/moments?
2. If a channel, such as $F_z$, looks incorrect or offset from expected values, what could a source of error be?


# Your Answers Go Here

---
## **Question 2: Filter Force and Moment Signals**

Evaluate the impact of cutoff frequency on one of your moment signals from Question 1. Then use a zero-phase-shift Butterworth filter on all calculated force/moment signals from Question 1.

Use the provided functions to run spectral analysis on and filter all of your force and moment data for both EO and EC conditions. As a reminder, this function loops through each column of the input data frame that is not labeled as "Time," calculates a unique cutoff frequency for that column, and then applies a zero-phase shift Butterworth filter to that column using the calculated cutoff frequency. Then, create two 6x1 subplots showing each filtered signal for both the EO and EC conditions.

**This filtered force/moment data is what you will use for the remainder of the questions.**


The following helper functions will be used throughout your analysis:
- `spectral_analysis()` - Performs power spectral density analysis to determine optimal filter cutoff frequencies
- `filter_timeseries_data()` - Applies a low-pass Butterworth filter to remove high-frequency noise

Run the cell below to define these functions.

In [ ]:
def spectral_analysis(
    data, column_name, sampling_freq, nperseg=None, window="hann", threshold=99, plot=False
):
    """
    Perform spectral analysis on the given data using the Welch method. This function computes the power spectral density and cumulative power spectrum, optionally plotting the results. Cutoff frequency is selected based on a cumulative power threshold.

    Args:
        data (array-like): The input signal data for which spectral analysis is to be performed.
        column_name (str): The name of the data used for plotting.
        sampling_freq (float): The sampling frequency of the input data.
        nperseg (int, optional): Length of each segment for the Welch method. If None, defaults to the length of the data.
        window (str, optional): The window function to use. Defaults to 'hann'.
        threshold (float): The percentage threshold for determining the cutoff frequency in the cumulative power spectrum.
        plot (bool, optional): If True, generates plots for the power spectral density and cumulative power spectrum. Defaults to False.

    Returns:
        float: The frequency at which the cumulative power spectrum exceeds the specified threshold.

    Raises:
        ValueError: If the input data is empty or if the sampling frequency is non-positive.

    Examples:
        >>> freq = spectral_analysis(data, 1000, threshold=95, plot=True)
    """
    data = np.asarray(data)

    if len(data) == 0:
        raise ValueError("Input data is empty")

    if sampling_freq <= 0:
        raise ValueError("Sampling frequency must be positive")

    if nperseg is None:
        nperseg = len(data)

    f, Pxx = welch(data, fs=sampling_freq, nperseg=nperseg, window=window)

    # Calculate cumulative power spectrum
    cumulative_power = np.cumsum(Pxx)

    # Normalize cumulative power to get a percentage
    cumulative_power_percent = cumulative_power / cumulative_power[-1] * 100

    # Find the index where the cumulative power levels off
    cutoff_index = np.argmax(
        cumulative_power_percent > threshold
    )  # Use the specified threshold

    if plot == True and column_name != "Time":
        # Plot the results
        plt.figure(figsize=(12, 6), dpi=100)
        plt.suptitle(f'Spectral Analysis for: {column_name}')
        # Plot Power Spectral Density
        plt.subplot(2, 1, 1)
        plt.semilogy(f, Pxx)
        plt.title('Power Spectral Density')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power/Frequency (dB/Hz)')
        plt.grid(True)

        # Plot Cumulative Power Spectrum
        plt.subplot(2, 1, 2)
        plt.plot(f, cumulative_power_percent)
        plt.axvline(x=f[cutoff_index], color='r', linestyle='--',
                    label=f'Cutoff Frequency: {f[cutoff_index]:.2f} Hz')
        plt.title('Cumulative Power Spectrum')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Cumulative Power (%)')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()
    return f[cutoff_index]


def filter_timeseries_data(data, sampling_freq, custom_cutoff_frequency=None, threshold=99, plot=False):
    """
    Filter time series data using a low-pass Butterworth filter. This function applies a moving average smoothing filter before the Butterworth filter to each column of the input data besides the column named 'Time', based on a specified or calculated cutoff frequency.

    Args:
        data (DataFrame): The input time series data to be filtered. Must be a pandas DataFrame.
        sampling_freq (float): The sampling frequency of the input data.
        custom_cutoff_frequency (float, optional): A user-defined cutoff frequency for the filter. If not provided, it will be determined using spectral analysis.
        threshold (float): Cumulative power threshold for automatic cutoff determination. Defaults to 99.
        plot (bool): Whether to plot spectral analysis results. Defaults to False.

    Returns:
        DataFrame: The filtered time series data, with the same structure as the input.

    Raises:
        ValueError: If the input data is not a pandas DataFrame.

    Examples:
        >>> filtered_data = filter_timeseries_data(data, sampling_freq=1000, threshold=95, plot=False)
    """
    if not isinstance(data, (pd.DataFrame)):
        raise ValueError(
            "Unsupported data type. Please provide a pandas DataFrame.")

    fs = float(sampling_freq)

    def apply_filter(column, cutoff_frequency):
        column = np.asarray(column)
        b, a = butter(N=4, Wn=cutoff_frequency,
                      btype="low", fs=fs, output="ba")
        return filtfilt(b, a, column)

    if custom_cutoff_frequency is None:
        cutoff_frequencies = {column: float(spectral_analysis(
            data[column], column, sampling_freq, threshold=threshold, plot=plot)) for column in data.columns}
    else:
        cutoff_frequencies = {
            column: custom_cutoff_frequency for column in data.columns}

    filtered_data = data.apply(lambda column: apply_filter(
        column, cutoff_frequencies[column.name]) if column.name != "Time" else column)

    return filtered_data

Now let's perform spectral analysis to calculate cutoff frequencies for each component. Decide on a cumulative power threshold for filtering and assign it in the cell below.

In [ ]:
# 1. Run spectral analysis on each force/moment component to determine cutoff frequencies
f_cutoffs_eo = {}
f_cutoffs_ec = {}
for column in ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']:
    if column != 'Time':
        print(f"Analyzing {column} for EO condition...")
        f_cutoffs_eo[column] = spectral_analysis(
            eo_data[column], column, sampling_freq=1000, threshold=#TODO, plot=True)
        print(f"Analyzing {column} for EC condition...")
        f_cutoffs_ec[column] = spectral_analysis(
            ec_data[column], column, sampling_freq=1000, threshold=#TODO, plot=True)

# 2. Print the cutoff frequencies for EO and EC conditions
print("Cutoff Frequencies for EO Condition:")
for key, value in f_cutoffs_eo.items():
    print(f"{key}: {value:.2f} Hz")
print("\nCutoff Frequencies for EC Condition:")
for key, value in f_cutoffs_ec.items():
    print(f"{key}: {value:.2f} Hz")

Now let's use our provided filtering function to perform low-pass filtering on our data based on spectral analysis. Use the same cumulative power threshold as you picked above.

In [ ]:
# 3. Filter the EO and EC data using the determined cutoff frequencies
filtered_eo = filter_timeseries_data(
    eo_data[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']],
    sampling_freq=1000,
    threshold=#TODO,
    plot=False
)
filtered_ec = filter_timeseries_data(
    ec_data[['Time', 'Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']],
    sampling_freq=1000,
    threshold=#TODO,
    plot=False,
)

for col in ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']:
    eo_data[f'{col}_filtered'] = filtered_eo[col]
    ec_data[f'{col}_filtered'] = filtered_ec[col]

Plot the filtered vs unfiltered data to see differences

In [ ]:
# 4. Create two 6x1 subplots comparing filtered vs unfiltered data for EO and EC conditions
fig, axes = plt.subplots(6, 1, figsize=(10, 8), dpi=100, sharex=True)
force_labels = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']
for i, label in enumerate(force_labels):
    axes[i].plot(eo_data['Time'], eo_data[label],
                 label='Unfiltered EO', color='tab:gray', alpha=0.5)
    axes[i].plot(eo_data['Time'], eo_data[label + '_filtered'],
                 label='Filtered EO', color='tab:blue')
    axes[i].set_title(f'EO Condition: {label} - Filtered vs Unfiltered')
    axes[i].set_ylabel(label)
    axes[i].legend()
plt.xlabel('Time')
plt.tight_layout()
plt.show()
fig, axes = plt.subplots(6, 1, figsize=(10, 8), dpi=100, sharex=True)
for i, label in enumerate(force_labels):
    axes[i].plot(ec_data['Time'], ec_data[label],
                 label='Unfiltered EC', color='tab:gray', alpha=0.5)
    axes[i].plot(ec_data['Time'], ec_data[label + '_filtered'],
                 label='Filtered EC', color='tab:orange')
    axes[i].set_title(f'EC Condition: {label} - Filtered vs Unfiltered')
    axes[i].set_ylabel(label)
    axes[i].legend()
plt.xlabel('Time')
plt.tight_layout()
plt.show()

### Discussion Questions
1. What cumulative power threshold did you pick for spectral analysis? Justify your choice.
2. How could your chosen cutoff frequency affect the balance metrics later in the notebook?
3. If EO and EC conditions yield different cutoff frequencies, what does that imply about the differences in sway patterns between the conditions?

# Your Answers Go Here

---
## **Question 3: Calculate and Plot Center of Pressure (CoP)**

Calculate the center of pressure (CoP) location and create a 1x2 subplot to show the CoP location (x,y) for all time points (stabilogram) for the EO and EC conditions. Use your filtered force/moment data from Question 2.

### Instructions: Calculating CoP Location

To find the location of the Center of Pressure (CoP), you need information about the force plate's shear center relative to its geometric center. The difference between the shear center and geometric center are negligible in the $x$ and $y$ directions, so we only consider the $z$ direction. For this specific force plate:

$$
z_0 = -37.645 \text{ mm}
$$

Assuming at a certain time-point $t_i$ we have the 1×6 row vector with forces and moments $\vec{Y_i}=(F_{x,i},F_{y,i},F_{z,i},M_{x,i},M_{y,i},M_{z,i})$, we can calculate the CoP location using:

$$
CoP_{x,i}=\frac{M_{y,i}-z_0F_{x,i}}{F_{z,i}}
$$

$$
CoP_{y,i}=\frac{M_{x,i}+z_0F_{y,i}}{F_{z,i}}
$$

**Important:** Be careful with the moment units and the units of $z_0$. They must be consistent for your CoP calculations.

### Creating the Stabilogram Plot

Before making your 1×2 stabilogram subplot:
- **Center the CoP data** by subtracting the mean $\overline{CoP_x}$ and $\overline{CoP_y}$ from each time point so the stabilogram is centered at the origin.
- Select the same axis limits to more easily compare the two conditions
- Clearly label your axes using appropriate anatomical conventions


Let's first start by calculating $\text{CoP}_x$ and $\text{CoP}_y$ for each condition

In [ ]:
# 1. Calculate CoP x and y positions using the equations above
z_0 = -37.645*1e-3  # (m) Shear center relative to geometric center
COP_EO = pd.DataFrame(columns=['COPx', 'COPy']) # Create empty DataFrame to store CoP results for EO condition
COP_EC = pd.DataFrame(columns=['COPx', 'COPy']) # Create empty DataFrame to store CoP results for EC condition

#TODO - Calculate COP_EO and COP_EC using the equations provided, utilizing the filtered force and moment data from eo_data and ec_data respectively. Use the empty dataframes created above to store the results.

Center the CoP data about the origin

In [ ]:
# 2. Center the CoP data about the origin
#TODO - Center the CoP data by subtracting the mean from each column.

Create stabilogram plots for comparing the two conditions

In [ ]:
# 3. Create stabilogram plots comparing EO and EC conditions
fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=100) # Create a 1x2 subplot for EO and EC conditions

# Determine maximum range for centering plots about origin
x_max = #TODO add code to calculate x_max using the maximum absolute values of the centered CoP x positions from both EO and EC conditions, multiplied by 1.1 for padding
y_max = #TODO add code to calculate y_max using the maximum absolute values of the centered CoP y positions from both EO and EC conditions, multiplied by 1.1 for padding

max_range = max(x_max, y_max)  # Ensure equal scaling in both axes

# First subplot for EO condition
axes[0].plot(#TODO,#TODO, color='tab:blue') # add code to finish this line to plot the centered CoP x and y positions for EO condition
# TODO set the plot title
# TODO set the x and y labels. Hint: make sure to include the units in the labels
# TODO set the x and y limits to be centered about the origin using max_range
axes[0].set_aspect('equal', adjustable='box') # This ensures that the x and y axes have the same scale, which is important for accurately interpreting the sway patterns in the stabilogram.
axes[0].grid(True) # Add grid lines

# Second subplot for EC condition
# TODO create the second subplot for EC condition by plotting the centered CoP x and y positions for EC condition, setting the title, labels, limits, aspect ratio, and grid lines similar to the EO subplot above. Use axes[1] for the EC subplot.
axes[1].plot(#TODO,#TODO, color='tab:orange') # add code to finish this line to plot the centered CoP x and y positions for EO condition

plt.tight_layout() # Adjust layout to prevent overlap
plt.show() # Display the stabilogram plots

### Discussion Questions:

1. Why might CoP be more sensitive than CoM for detecting balance changes?
2. Why is centering the CoP data important before comparing EO and EC stabilograms?

# Your Answers Go Here

---
## **Question 4: Calculate Balance Metrics**

Calculate and report the following balance metrics for the EO and EC conditions using the centered data:

- **Anterior-posterior (AP) excursion range**

    $$ \text{CoP}_{AP,max} - \text{CoP}_{AP,min} $$

- **Medial-lateral (ML) excursion range**

    $$ \text{CoP}_{ML,max} - \text{CoP}_{ML,min} $$

- **Path length**

    $$
    \text{Path Length}=\sum_{i=2}^{N}\sqrt{(\text{CoP}_{x,i}-\text{CoP}_{x,i-1})^2+(\text{CoP}_{y,i}-\text{CoP}_{y,i-1})^2}
    $$

- **Average CoP speed**

    $$
    \frac{\text{path length}}{T}
    $$

    where $T$ is the total time

- **Mean excursion distance**

    $$
    \begin{aligned} 
    d_i&=\sqrt{\text{CoP}_{x,i}^2+\text{CoP}_{y,i}^2} \\
    \bar{d}&=\frac{1}{N}\sum_{i=1}^{N}d_i
    \end{aligned}
    $$

- **Standard deviation of excursion distance**

    $$
    \begin{aligned}
    \sigma_d&=\sqrt{\frac{1}{N-1}\sum_{i=1}^{N}(d_i-\bar{d})^2}
    \end{aligned}
    $$

- **Angular deviation from AP axis**

    $$
    \theta_i=\arctan\left(\frac{\Delta \text{CoP}_{y,i}}{\Delta \text{CoP}_{x,i}}\right), \quad
    \text{Angular Deviation}=\frac{1}{N-1}\sum_{i=1}^{N-1}\left|\theta_i\right|\cdot\frac{180}{\pi}
    $$

    Note: make sure to use the math.atan2() or np.atan2() function to automatically adjust for quadrant

- **Area of 95% confidence ellipse**

    $$
    A_{ellipse,95\%} = \pi \cdot a \cdot b = \pi \cdot 2\sqrt{5.991 \lambda_1} \cdot 2\sqrt{5.991 \lambda_2}
    $$

    where $\lambda_1$ and $\lambda_2$ are the eigenvalues of the covariance matrix of the centered CoP data, and $a$, $b$ are the semi-major and semi-minor axes, respectively.


In [ ]:
# 1. Calculate the following CoP metrics for both EO and EC conditions

# Initialize dictionaries to store CoP metrics for EO and EC conditions
EO_metrics = {
    "AP_Excursion_Range": None,
    "ML_Excursion_Range": None,
    "Path_Length": None,
    "Average_CoP_Speed": None,
    "Mean_Excursion_Distance": None,
    "Std_Excursion_Distance": None,
    "Angular_Deviation": None,
    "Ellipse_Area_95": None,
}
EC_metrics = {
    "AP_Excursion_Range": None,
    "ML_Excursion_Range": None,
    "Path_Length": None,
    "Average_CoP_Speed": None,
    "Mean_Excursion_Distance": None,
    "Std_Excursion_Distance": None,
    "Angular_Deviation": None,
    "Ellipse_Area_95": None,
}

# Anterior-Posterior Excursion Range
# TODO add code to calculate based on the equations above and assign to EO_metrics and EC_metrics dictionaries

# Medial-Lateral Excursion Range
# TODO add code to calculate based on the equations above and assign to EO_metrics and EC_metrics dictionaries

# Path Length
# TODO add code to calculate based on the equations above and assign to EO_metrics and EC_metrics dictionaries

# Average CoP Speed
# TODO add code to calculate based on the equations above and assign to EO_metrics and EC_metrics dictionaries

# Mean Excursion Distance
# TODO add code to calculate based on the equations above and assign to EO_metrics and EC_metrics dictionaries

# Standard Deviation of Excursion Distance
# TODO add code to calculate based on the equations above and assign to EO_metrics and EC_metrics dictionaries

# Angular Deviation from Anterior-Posterior Axis
def angular_deviation(cop_data):
    """
    Calculate the mean angular deviation of CoP trajectory from the anterior-posterior (AP) axis.

    Args:
        cop_data (DataFrame): DataFrame containing 'COPx_centered' and 'COPy_centered' columns.

    Returns:
        float: Mean absolute angular deviation in degrees from the AP axis (y-axis).
    """
    delta_x = np.diff(cop_data['COPx_centered'])
    delta_y = np.diff(cop_data['COPy_centered'])
    #TODO add code to calculate the angles of the CoP trajectory segments relative to the AP axis using arctan2, then take the absolute value and convert to degrees. Finally, return the mean.
    return 


EO_metrics['Angular_Deviation'] = angular_deviation(COP_EO)
EC_metrics['Angular_Deviation'] = angular_deviation(COP_EC)

# Area of 95% Confidence Ellipse. This code has been provided for you. There is nothing to complete here.
def ellipse_area(cop_data):
    """
    Calculate the area of a 95% confidence ellipse from CoP data.

    Args:
        cop_data (DataFrame): DataFrame containing COPx_centered and COPy_centered columns

    Returns:
        float: Area of the 95% confidence ellipse
    """
    cov_matrix = np.cov(cop_data['COPx_centered'],
                        cop_data['COPy_centered'])  # Covariance matrix
    eigenvalues, _ = np.linalg.eig(cov_matrix)  # Eigenvalues and eigenvectors
    a = 2 * np.sqrt(5.991 * eigenvalues[0])  # Semi-major axis 95% confidence
    b = 2 * np.sqrt(5.991 * eigenvalues[1])  # Semi-minor axis 95% confidence
    return np.pi * a * b


# Calculate ellipse areas
EO_metrics['Ellipse_Area_95'] = ellipse_area(COP_EO)
EC_metrics['Ellipse_Area_95'] = ellipse_area(COP_EC)

# Print the calculated CoP metrics for both EO and EC conditions
print("CoP Metrics for EO Condition:")
for key, value in EO_metrics.items():
    print(f"{key}: {value:.2f}")
print("\nCoP Metrics for EC Condition:")
for key, value in EC_metrics.items():
    print(f"{key}: {value:.2f}")

Now let's calculate the percent change of each metric between the two conditions

In [ ]:
# Calculate and print the percentage changes between EO and EC conditions for each metric
print("\nPercentage Changes from EO to EC Condition:")
for key in EO_metrics.keys():
    percent_change = #TODO add code to calculate the percentage change from EO to EC for each metric using the formula: ((EC - EO) / EO) * 100
    print(f"{key}: {percent_change:.2f}%")

### Discussion Questions:

1. Which metrics changed the most between the two conditions? Which changed the least? Explain what differences in these metrics imply about the CoP trajectory.
2. There are many CoP metrics that have been reported in balance literature. Find a metric that we didn't use and define it. Please also provide the source in APA format.

# Your Answers Go Here

---

## Export Notebook as HTML for Submission

Run the cells below to export this notebook as an HTML file for submission to Canvas.

**Submission Requirements:**
- Upload both the `.ipynb` file and the exported `.html` file to Canvas
- Open the HTML file before submitting to confirm everything is readable and appears as expected
- Ensure all code cells have been run and all outputs are visible

In [ ]:
# Helper function to export notebook as HTML
def export_current_notebook(notebook_path, output_filename="exported_notebook.html"):
    """
    Exports the specified Colab notebook (with all cell outputs) as an HTML file.

    Parameters:
    - notebook_path (str): The full path to the notebook file to export.
    - output_filename (str): The name of the output HTML file (default is "exported_notebook.html").

    This function reads the notebook file, converts it to an HTML file,
    and saves the output with the given filename. It also initiates a download of the HTML file.
    """
    try:
        # Read the notebook file
        with open(notebook_path, "r", encoding="utf-8") as f:
            notebook_content = nbformat.read(f, as_version=4)

        # Convert notebook to HTML
        html_exporter = HTMLExporter()

        # Ensure that input and output cells are included in the exported HTML
        html_exporter.exclude_input = False  # Include code input
        html_exporter.exclude_output = False  # Ensure that outputs are captured

        # Export the notebook to HTML
        html_data, _ = html_exporter.from_notebook_node(notebook_content)

        # Add CSS to wrap lines and prevent clipping
        wrap_css = """
        <style>
            pre, code {
                word-wrap: break-word;
                white-space: pre-wrap;
                word-break: break-word;
            }
        </style>
        """

        # Insert the CSS at the top of the HTML document
        html_data = wrap_css + html_data

        # Save the HTML file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_data)

        print(f"Notebook exported successfully as {output_filename}")

        # Download the file
        files.download(output_filename)

    except Exception as e:
        print(f"An error occurred: {e}")


# Update these paths to match your notebook location and desired output name
# Change to your notebook path
# TODO: Update this path to point to your notebook in your Google Drive. You can right-click on the notebook file in the Colab file tree and select "Copy path" or "Copy relative path" to get the correct path. Make sure your NetID is included in the filename.
notebook_to_export = '/content/drive/MyDrive/ME481-BalanceLab/BalancePostLabIndividualAssignment-NetID.ipynb'

# This will save to your downloads folder
output_filename = 'BalancePostLabIndividualAssignment-NetID.html'

# Export the notebook
export_current_notebook(notebook_path=notebook_to_export,
                        output_filename=output_filename)